# hello

In [ ]:
import pandas as pd
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

df = movies.merge(credits,on='title')

In [ ]:
import ast
def safe_to_list(obj):
    if pd.isna(obj):
        return []
    if isinstance(obj,list):
        return obj
    if isinstance(obj,str):
        v = ast.literal_eval(obj)
        if isinstance(v,list):
            return v
        else:
            [obj]            
    return [obj]        

def get_names(obj):
    names = []
    if not isinstance(obj,list):
        return ""
    for item in obj:
        names.append(item['name'])
    
    if isinstance(names,list):
        return names
    else:
        return "" 
        
def get_director(obj):
    if isinstance(obj,list):
       try:
            for item in obj:
              if item['job']=='Director':
                 return item['name']
       except Exception:
           return ""       

def top_cast(obj):
    names = []
    count = 0
    if isinstance(obj,list):
        try:
            for item in obj:
                if count != 3:
                    names.append(item)
                    count+=1
                else:
                    break
        except Exception:
            return []            
    return names 

In [ ]:
cols_to_list  = ['genres','cast','crew','keywords']
for c in cols_to_list:
    prased = c + '_names'
    df[prased] = df[c].apply(safe_to_list)

In [ ]:
df['genres_names'] = df['genres_names'].apply(get_names)
df['cast_names'] = df['cast_names'].apply(get_names)
df['keywords_names'] = df['keywords_names'].apply(get_names)

In [ ]:
df['top_cast'] = df['cast_names'].apply(top_cast)

In [ ]:
df['top_cast'].head()

In [ ]:
df['director'] = df['crew_names'].apply(get_director)

In [ ]:
df['director']

In [ ]:
df['title_clean'] = df['title'].astype(str).str.lower().str.strip()

In [ ]:
df[['title_clean','title']].head()

In [ ]:
def join_list(obj):
    if not isinstance(obj,list):
        return ""
    return " ".join([str(x) for x in obj])

def build_text(row):
    parts = []

    parts.append(join_list(row['genres_names']))
    parts.append(join_list(row['top_cast']))
    parts.append(join_list(row['keywords_names']))

    if isinstance(row['director'],str):
        parts.append(row['director'])
    if isinstance(row['overview'],str):
        parts.append(row['overview']) 

    return " ".join(parts)           
    

In [ ]:
df['text_for_bert'] = df.apply(build_text,axis=1)

In [ ]:
df.iloc[0]['text_for_bert']

In [ ]:
# !pip install -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = 'all-MiniLm-L6-v2'
bert_model = SentenceTransformer(model_name)
bert_model.max_seq_length = 64

In [ ]:
texts = df['text_for_bert'].fillna("").tolist()
len(texts)

In [ ]:
texts[0][:500]

link for knowing about this model's fuctions brefiely 
https://chatgpt.com/s/t_69203ac05cac81919e82734fc39cb8c4

In [ ]:
# embedding = bert_model.encode(
#     texts,
#     batch_size=4,
#     show_progress_bar=True,
#     convert_to_numpy=True,
#     normalize_embeddings=True
# )
# import joblib
# joblib.dump(embedding,'bert_embedding.joblib')

In [ ]:
indices = pd.Series(df.index , index=df['title_clean']).drop_duplicates()

In [ ]:
def clean_title(t):
    return str(t).lower().strip()

this link give u a brief intro of Dot product
https://chatgpt.com/s/t_69204422d4e081919e1ff53eaf5affb6

In [ ]:
from difflib import get_close_matches

def recommender(title,topn=10):
    if not isinstance(title,str):
        raise ValueError('give a proper String')
    
    title_key = clean_title(title)

    if title_key not in indices:
        choices = get_close_matches(title_key,df['title_clean'].tolist , n=6 , cutoff=0.59)
        if choices == [] or "":
            return None , choices
        return f"{title_key} is not found but here are some options->",choices
    
    idx = indices[title_key]
    query_emb = embedding[idx]
    sim_score = np.dot(embedding,query_emb)
    top_indices = np.argsort(sim_score)[::-1]
    top_indices = top_indices[top_indices != idx]
    top_indices = top_indices[:topn]
    top_socre = sim_score[top_indices]

    results = df.iloc[top_indices][['title','genres_names','director']].copy()
    results['score'] = top_socre
    results = results.reset_index(drop=True)

    return f"recommendations for {df.loc[idx,'title']}",results



In [ ]:
display(recommender('Avatar',topn=2))

In [ ]:
df.to_csv('cleaned_movies_data.csv',index=False)

np.save('bert_embedding.npy',embedding)

indices_dict = indices.to_dict()
joblib.dump(indices_dict,'indices_dict.joblib')
